In [1]:
#Import packages

#Rdkit
from rdkit import Chem

#Scipy
from scipy import stats 
from scipy.stats import uniform,ranksums 

#Pandas
import pandas as pd
from pandas import DataFrame

#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc

#Others
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import joblib #jbolib模块
import random
import numpy as np
import os
import math
import warnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from plotnine import *

In [2]:
import torch
import deepchem as dc
import numpy as np

In [3]:
def print_RxC(dataframe):
    rows= dataframe.shape[0]
    columns= dataframe.shape[1]
    rows_columns=print(dataframe.shape[0],"Moleculars ×",dataframe.shape[1],"Descriptors")
    return rows_columns
def trimm_correlated(df_in, threshold):
    df_corr = df_in.corr(method='spearman')
    df_not_correlated = ~(df_corr.mask(np.tril(np.ones([len(df_corr)]*2, dtype=bool))).abs() > threshold).any()
    un_corr_idx = df_not_correlated.loc[df_not_correlated[df_not_correlated.index] == True].index
    df_out = df_in[un_corr_idx]
    return df_out

In [4]:
#Setting the Work Path
warnings.filterwarnings ('ignore')
#Work Path
os.chdir("C:\\Users\\A\\Desktop\\Bioactive\\TNF-α")

In [21]:
df = pd.read_csv('./pubchem_protacxn_P01375_bioactivity_protein.csv',index_col=0)

In [22]:
df

,activity,aid,sid,cid,geneid,pmid,aidtype,aidmdate,hasdrc,rnai,...,aidsrcname,aidname,cmpdname,targetname,targeturl,ecs,repacxn,taxid,cellids,targettaxid
baid,,,,,,,,,,,,,,,,,,,,,
63471797,Active,1852,3711304,2083077.0,7124,NaN,Other,20101230,0,0,...,Burnham Center for Chemical Genomics,HTS assay for identification of inhibitors of ...,"1-[1-(4-Methoxyphenyl)-2,5-dimethyl-3-pyrrolyl...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN
63471798,Active,1852,3712888,2082098.0,7124,NaN,Other,20101230,0,0,...,Burnham Center for Chemical Genomics,HTS assay for identification of inhibitors of ...,5-methyl-N-quinolin-5-ylthiophene-2-sulfonamide,TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN
63471863,Active,1852,4244047,2158903.0,7124,NaN,Other,20101230,0,0,...,Burnham Center for Chemical Genomics,HTS assay for identification of inhibitors of ...,"N-[5-(4-fluorophenyl)-1,3,4-oxadiazol-2-yl]-4-...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN
63471864,Active,1852,4247289,3241516.0,7124,NaN,Other,20101230,0,0,...,Burnham Center for Chemical Genomics,HTS assay for identification of inhibitors of ...,"N-(5-(2-chlorophenyl)-1,3,4-oxadiazol-2-yl)-4-...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN
63471865,Active,1852,4249058,2158895.0,7124,NaN,Other,20101230,0,0,...,Burnham Center for Chemical Genomics,HTS assay for identification of inhibitors of ...,4-(azepan-1-ylsulfonyl)-N-[5-(2-chlorophenyl)-...,TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
381121614,Unspecified,1806616,471699363,165112643.0,7124,NaN,Confirmatory,20230307,0,0,...,BindingDB,TNF-alpha RELEASE from US Patent US11365204: '...,NaN,TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN
381121678,Unspecified,1806616,471699358,138491955.0,7124,NaN,Confirmatory,20230307,0,0,...,BindingDB,TNF-alpha RELEASE from US Patent US11365204: '...,[1-[(5R)-4-(2-oxaspiro[3.3]heptan-6-ylamino)-5...,TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN
381121736,Unspecified,1806616,471699301,138502099.0,7124,NaN,Confirmatory,20230307,0,0,...,BindingDB,TNF-alpha RELEASE from US Patent US11365204: '...,"[1-[4-[Oxan-4-yl(propyl)amino]-5-oxo-6,7-dihyd...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN


In [23]:
df.index=df['cid']
df.dropna(subset=["cid"], inplace=True)  
df['Activity']=np.where(df['activity']=='Active', 1, 0)
df.sort_values(by="Activity" , inplace=True, ascending=False) 
df.drop_duplicates(subset='cid', keep='first', inplace=True)
df

,activity,aid,sid,cid,geneid,pmid,aidtype,aidmdate,hasdrc,rnai,...,aidname,cmpdname,targetname,targeturl,ecs,repacxn,taxid,cellids,targettaxid,Activity
cid,,,,,,,,,,,,,,,,,,,,,
2083077.0,Active,1852,3711304,2083077.0,7124,NaN,Other,20101230,0,0,...,HTS assay for identification of inhibitors of ...,"1-[1-(4-Methoxyphenyl)-2,5-dimethyl-3-pyrrolyl...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN,1
138703696.0,Active,1803711,406798511,138703696.0,7124,NaN,Confirmatory,20230307,0,0,...,IRAK4 Monocyte TNFalpha Cell Based Assay from ...,"(R)-6-(3-cyanopyrrolo[1,2-b]pyridazin-7-yl)-N-...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN,1
139316163.0,Active,1803711,406798479,139316163.0,7124,NaN,Confirmatory,20230307,0,0,...,IRAK4 Monocyte TNFalpha Cell Based Assay from ...,"((1R,3r)-3-((2-(3-cyano-pyrrolo[1,2-b]pyridazi...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN,1
139315919.0,Active,1803711,406798497,139315919.0,7124,NaN,Confirmatory,20230307,0,0,...,IRAK4 Monocyte TNFalpha Cell Based Assay from ...,"(R)-methyl ((3-((2-(3-cyanopyrrolo[1,2-b]pyrid...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN,1
138703798.0,Active,1803711,406798016,138703798.0,7124,NaN,Confirmatory,20230307,0,0,...,IRAK4 Monocyte TNFalpha Cell Based Assay from ...,"6-(3-cyanopyrrolo[1,2-b] pyridazin-7-yl)-N-((R...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2158020.0,Inactive,1852,17515153,2158020.0,7124,NaN,Other,20101230,0,0,...,HTS assay for identification of inhibitors of ...,N-[2-(5-chloro-2-methyl-1H-indol-3-yl)ethyl]-N...,TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN,0
9631088.0,Inactive,1852,22410794,9631088.0,7124,NaN,Other,20101230,0,0,...,HTS assay for identification of inhibitors of ...,3-[(2E)-2-(thiophen-2-ylmethylidene)hydrazinyl...,TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN,0
3247362.0,Inactive,1852,24707790,3247362.0,7124,NaN,Other,20101230,0,0,...,HTS assay for identification of inhibitors of ...,"N-[(R)-[(1R,2S)-1-methyl-2-[(2R)-1-(naphthalen...",TNF - tumor necrosis factor (human),/gene/7124,NaN,P01375,9606,NaN,NaN,0


In [24]:
np.savetxt('./cid.txt',df['cid'],fmt='%d')

In [26]:
data1 = pd.read_csv('./data_1.csv',index_col=0)
data2 = pd.read_csv('./data_2.csv',index_col=0)
data= pd.concat([data1,data2 ])

In [29]:
df['IsomericSMILES']=data['IsomericSMILES']

In [30]:

Original_data=pd.DataFrame(df['IsomericSMILES'])
Original_data['Activity']=df['Activity']
Chemdraw_smiles=Original_data['IsomericSMILES'].tolist()

Original_data.head()

,IsomericSMILES,Activity
cid,,
2083077.0,CC1=CC(=C(N1C2=CC=C(C=C2)OC)C)C(=O)CSC3=NN=C(O...,1
138703696.0,CC(C)([C@@H](CNC(=O)C1=CN=C(C=C1NC2=CN=C(C=C2)...,1
139316163.0,CC(C)([C@@H](CNC(=O)C1=CN=C(C=C1NC2CC(C2)COC(=...,1
139315919.0,CC(C)([C@@H](CNC(=O)C1=CN=C(C=C1NC2CC(C2)(CNC(...,1
138703798.0,CC(C)([C@@H](CNC(=O)C1=CN=C(C=C1NC2C[C@@H]3CNC...,1


In [31]:
Smis=[]
for i in Chemdraw_smiles:
    Smis.append(i)
Mols = []
for Smi in Smis:
    Mol = Chem.MolFromSmiles(Smi)
    Mols.append(Mol)
Canonical_smis = []
for Mol in Mols:
    Canonical_smi = Chem.MolToSmiles(Mol)
    Canonical_smis.append(Canonical_smi)
Original_data['Canonical_smiles']=Canonical_smis
Original_data.to_csv("./Original_data.csv",sep=',',header=1,index=1)
Original_data

,IsomericSMILES,Activity,Canonical_smiles
cid,,,
2083077.0,CC1=CC(=C(N1C2=CC=C(C=C2)OC)C)C(=O)CSC3=NN=C(O...,1,COc1ccc(-n2c(C)cc(C(=O)CSc3nnc(-c4cccs4)o3)c2C...
138703696.0,CC(C)([C@@H](CNC(=O)C1=CN=C(C=C1NC2=CN=C(C=C2)...,1,CC(C)(O)[C@H](F)CNC(=O)c1cnc(-c2ccc3cc(C#N)cnn...
139316163.0,CC(C)([C@@H](CNC(=O)C1=CN=C(C=C1NC2CC(C2)COC(=...,1,CC(C)(O)[C@H](F)CNC(=O)c1cnc(-c2ccc3cc(C#N)cnn...
139315919.0,CC(C)([C@@H](CNC(=O)C1=CN=C(C=C1NC2CC(C2)(CNC(...,1,COC(=O)NCC1(F)CC(Nc2cc(-c3ccc4cc(C#N)cnn34)ncc...
138703798.0,CC(C)([C@@H](CNC(=O)C1=CN=C(C=C1NC2C[C@@H]3CNC...,1,CC(C)(O)[C@H](F)CNC(=O)c1cnc(-c2ccc3cc(C#N)cnn...
...,...,...,...
2158020.0,CC1=C(C2=C(N1)C=CC(=C2)Cl)CCN(CC3=CC(=C(C(=C3)...,0,CNC(=S)N(CCc1c(C)[nH]c2ccc(Cl)cc12)Cc1cc(OC)c(...
9631088.0,C1=CC(=CC(=C1)N/N=C/C2=CC=CS2)C(=O)O,0,O=C(O)c1cccc(N/N=C/c2cccs2)c1
3247362.0,C[C@H]([C@@H]1C[C@@]1(C)[C@H](C2=CC=CC=C2)NC(=...,0,C[C@@H](C(=O)Nc1ccc2ccccc2c1)[C@@H]1C[C@@]1(C)...


In [32]:
#AlvaDesc
from alvadesccliwrapper.alvadesc import AlvaDesc
#AlvaDesc Path(Linux default alvaDescCLI location)
aDesc = AlvaDesc('E:\\C++\\alvaDesc\\alvaDescCLI.exe') 

In [35]:
#Calculate the descriotors
aDesc.set_input_SMILES(Canonical_smis)
if not aDesc.calculate_descriptors('ALL'):
 print('Error: ' + aDesc.get_error())
else:
 Data_gelation=aDesc.get_output()

In [36]:
X_original_data=pd.DataFrame(Data_gelation)
Desc_names_names=aDesc.get_output_descriptors()
X_original_data.columns=Desc_names_names
X_original_data.index=Original_data.index

X_original_data.head()

,MW,AMW,Sv,Se,Sp,Si,Mv,Me,Mp,Mi,...,s1_numAroBonds,s2_numAroBonds,s3_numAroBonds,s4_numAroBonds,s34_size,s34_relSize,s34_phSize,s34_phRelSize,chiralMoment,chiralPhMoment
cid,,,,,,,,,,,,,,,,,,,,,
2083077.0,425.57,8.866042,32.7972,48.5089,34.7672,53.2847,0.683275,1.010602,0.724317,1.110098,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
138703696.0,526.58,8.493226,41.9133,63.3778,42.4378,70.2759,0.676021,1.022223,0.684481,1.133482,...,0.0,0.0,0.0,27.0,37.000000,0.948718,11.000000,0.282051,54.589376,16.822604
139316163.0,585.65,8.134028,46.0073,74.0467,46.5686,82.7424,0.638990,1.028426,0.646786,1.149200,...,0.0,0.0,0.0,16.0,40.000000,0.952381,11.000000,0.261905,59.757845,16.822604
139315919.0,541.62,7.965000,43.0972,69.6504,43.8697,77.9875,0.633782,1.024271,0.645143,1.146875,...,0.0,0.0,0.0,16.0,37.000000,0.948718,11.000000,0.282051,54.589376,16.822604
138703798.0,491.63,7.448939,41.2843,66.4831,43.0232,75.2290,0.625520,1.007320,0.651867,1.139833,...,0.0,0.0,0.0,16.0,33.333333,0.925926,8.333333,0.231481,49.318632,13.379088


In [37]:
#Remove descriptors with excessive NA values
Thresh =int(0.8*len(X_original_data))
X_NAomit_data=X_original_data.dropna(
    #thresh=Thresh, 
    axis=1) 
X_NAomit_data.head()
X_NAomit_data.to_csv("./X_NAomit_data.csv",sep=',',header=1,index=1)

In [38]:
#Ranksum test
Ranksums_data=X_NAomit_data
Ranksums_data["Activity"]=Original_data['Activity']
Notgelable=Ranksums_data.loc[Ranksums_data["Activity"] == 0]
Gelable=Ranksums_data.loc[Ranksums_data["Activity"] == 1]
Ranksums_result=[]
for i in list(Ranksums_data.columns):
 NG = Notgelable[i]
 G = Gelable[i]
 Ranksums_caculate=ranksums(NG, G, alternative='two-sided')
 Ranksums_result.append(Ranksums_caculate)
Ranksums_results = pd.DataFrame(Ranksums_result, columns=['Statistics','Pvalues'])
Ranksums_results.index=Ranksums_data.columns#
Ranksums_results = Ranksums_results.drop('Activity',axis = 0)
Ranksums_results.sort_values(by="Pvalues" , inplace=True, ascending=True)
Ranksums_results

,Statistics,Pvalues
MW,-41.912096,0.0
SpAbs_Dz(p),-40.560997,0.0
HyWi_Dz(p),-40.501949,0.0
H_Dz(p),-43.800194,0.0
AVS_Dz(p),-40.670602,0.0
...,...,...
F01[Cl-Br],0.000000,1.0
F01[Cl-Cl],0.000000,1.0
F01[F-X],0.000000,1.0
B02[F-Si],0.000000,1.0


In [39]:
#Ranksum test <0.05
Ranksums_marked = Ranksums_results[Ranksums_results['Pvalues'] <0.05]
print(Ranksums_marked.shape)
Ranksums_marked.head()

(1261, 2)


,Statistics,Pvalues
MW,-41.912096,0.0
SpAbs_Dz(p),-40.560997,0.0
HyWi_Dz(p),-40.501949,0.0
H_Dz(p),-43.800194,0.0
AVS_Dz(p),-40.670602,0.0


In [40]:
X_ranksums_data=X_NAomit_data[Ranksums_marked.index]
X_ranksums_data.head()

,MW,SpAbs_Dz(p),HyWi_Dz(p),H_Dz(p),AVS_Dz(p),Wi_Dz(p),SM6_Dz(m),SM5_Dz(m),SM4_Dz(m),SM3_Dz(m),...,nImidazoles,CATS2D_01_DA,nROCON,F10[C-Cl],B10[C-Cl],NaaS,H-048,SaaS,F06[N-S],Cl-089
cid,,,,,,,,,,,,,,,,,,,,,
2083077.0,425.57,366.241285,9.392252,119.347119,172.138561,2493.702236,28.905256,24.022420,19.334777,14.267157,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.534013,0.0,0.0
138703696.0,526.58,610.311858,10.316556,166.426749,284.862162,5549.540702,32.254702,26.869305,21.524336,16.051873,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0
139316163.0,585.65,889.201573,11.073742,153.617741,409.656940,8595.081384,33.893507,28.224491,22.627489,16.845901,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
139315919.0,541.62,697.875347,10.565958,145.879376,321.544148,6263.467979,32.804580,27.326537,21.891928,16.324594,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
138703798.0,491.63,538.798870,10.075883,144.506569,251.987968,4531.411974,31.636576,26.353680,21.112422,15.743148,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [41]:
#Scale data
Scaler = preprocessing.MinMaxScaler() #StandardScaler
Transformer =Scaler.fit(X_ranksums_data)
X_scaled_data=Transformer.transform(X_ranksums_data)
X_scaled_data =pd.DataFrame(X_scaled_data)
X_scaled_data.columns=X_ranksums_data.columns
X_scaled_data.index=Original_data.index
X_scaled_data.to_csv("./X_scaled_data.csv",sep=',',header=1,index=1)
joblib.dump(Transformer, './Scaler_transformer.pkl')
X_scaled_data.head()

,MW,SpAbs_Dz(p),HyWi_Dz(p),H_Dz(p),AVS_Dz(p),Wi_Dz(p),SM6_Dz(m),SM5_Dz(m),SM4_Dz(m),SM3_Dz(m),...,nImidazoles,CATS2D_01_DA,nROCON,F10[C-Cl],B10[C-Cl],NaaS,H-048,SaaS,F06[N-S],Cl-089
cid,,,,,,,,,,,,,,,,,,,,,
2083077.0,0.100131,0.013547,0.507587,0.063662,0.013296,0.001367,0.527579,0.525839,0.525696,0.519653,...,0.0,0.0,0.0,0.0,0.0,0.5,0.0625,0.419647,0.0,0.0
138703696.0,0.125832,0.022601,0.560922,0.087584,0.022005,0.003041,0.589174,0.588881,0.586344,0.586353,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0625,0.000000,0.0,0.0
139316163.0,0.140863,0.032947,0.604614,0.081076,0.031646,0.004710,0.619311,0.618891,0.616900,0.616028,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0
139315919.0,0.129659,0.025849,0.575313,0.077144,0.024839,0.003432,0.599286,0.599006,0.596526,0.596546,...,0.0,0.0,0.5,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0
138703798.0,0.116939,0.019948,0.547034,0.076446,0.019465,0.002483,0.577807,0.577463,0.574934,0.574815,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.000000,0.0,0.0


In [42]:
#Remove high correlation descriptors
X_cor_data=trimm_correlated(X_scaled_data,0.80)
X_cor_data.index=Original_data.index
X_cor_data.to_csv("./ML_data.csv",sep=',',header=1,index=1)
print_RxC(X_cor_data)
X_cor_data.head()

6175 Moleculars × 288 Descriptors


,MW,CATS2D_02_AL,CATS2D_06_AL,P_VSA_MR_5,SAscore,P_VSA_LogP_5,D/Dtr05,CATS2D_07_AL,Eta_betaS_A,F04[N-N],...,CATS2D_07_DD,F09[N-S],N-068,minaaS,N-067,nImidazoles,CATS2D_01_DA,nROCON,H-048,F06[N-S]
cid,,,,,,,,,,,,,,,,,,,,,
2083077.0,0.100131,0.366667,0.081967,0.021267,0.258958,0.174110,0.078366,0.108434,0.811912,0.000000,...,0.0,0.0,0.0,0.818675,0.0,0.0,0.0,0.0,0.0625,0.0
138703696.0,0.125832,0.433333,0.131148,0.092569,0.370407,0.505067,0.062902,0.048193,0.848485,0.555556,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0625,0.0
139316163.0,0.140863,0.366667,0.163934,0.081090,0.363123,0.494151,0.032011,0.084337,0.856061,0.555556,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.5,0.0000,0.0
139315919.0,0.129659,0.333333,0.163934,0.087125,0.361647,0.344681,0.029036,0.036145,0.840326,0.555556,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.5,0.0000,0.0
138703798.0,0.116939,0.333333,0.163934,0.088677,0.383732,0.292887,0.073900,0.084337,0.839646,0.555556,...,0.0,0.0,0.0,0.000000,0.2,0.0,0.0,0.0,0.0000,0.0
